# Interaccion de Postgresql y Python

## SQLAlchemy

## Psycopg2

Se genera:

1. Una conexion,
2. Un cursor
3. Las tareas

In [1]:
import psycopg2, sqlalchemy

conn = psycopg2.connect("dbname=postgres user=postgres password=2719")
print(conn.status)
cursor = conn.cursor()
cursor.execute('SELECT * FROM album;')
cursor.fetchall()